# NYC Airbnb Promotions Shortlist (2019)
## Python + Pandas Data Cleaning & Filtering Project

### Business goal
Airbnb is planning a new social media campaign in New York City. The goal is to identify listings that are likely to be attractive “best deals” and worth promoting.

### Promotion rules (shortlist criteria)
A listing is considered promotion-ready if it meets **all** of the following:
- **Price:** $100/night or less  
- **Minimum stay:** 2 nights or fewer  
- **Reviews:** at least 100 reviews  
- **Availability:** available at least 180 days per year  

### What this notebook delivers
- A cleaned dataset (removing missing and duplicate records)
- A filtered “promotion shortlist”
- A borough breakdown of shortlisted listings
- A separate luxury analysis focused on Brooklyn and Manhattan


## Setup

In [2]:
import pandas as pd
from pathlib import Path

## 1) Load the dataset and inspect data quality

I start by loading the NYC Airbnb 2019 dataset and checking:
- column types
- missing values
- overall row count


In [3]:
df = pd.read_csv("AB_NYC_2019.csv")

print("Preview (1 row):")
print(df.head(1).to_string(index=False))
print("\nDataset info:")
df.info()


Preview (1 row):
  id                               name  host_id host_name neighbourhood_group neighbourhood  latitude  longitude    room_type  price  minimum_nights  number_of_reviews last_review  reviews_per_month  calculated_host_listings_count  availability_365
2539 Clean & quiet apt home by the park     2787      John            Brooklyn    Kensington  40.64749  -73.97237 Private room    149               1                  9  2018-10-19               0.21                               6               365

Dataset info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48895 entries, 0 to 48894
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              48895 non-null  int64  
 1   name                            48879 non-null  object 
 2   host_id                         48895 non-null  int64  
 3   host_name                       48874 non-null  o

## 2) Missing data impact (who might get missed?)

Before filtering for promotions, it's useful to understand how many listings could be excluded purely because of incomplete data.

This is a common real-world issue: a listing might be good, but if key fields are missing, it can’t be evaluated against campaign rules.


In [4]:
df_clean = df.dropna()

print("Rows before dropna:", len(df))
print("Rows after dropna :", len(df_clean))
print("Potentially missed listings due to missing data:", len(df) - len(df_clean))


Rows before dropna: 48895
Rows after dropna : 38821
Potentially missed listings due to missing data: 10074


## 3) Select only the columns needed for the analysis

To keep the analysis focused, I remove columns that are not required for the promotion rules or reporting outputs.


In [5]:
# Columns not needed for this analysis (safe to remove for this project scope)
drop_cols = [
    "host_name",
    "neighbourhood",
    "latitude",
    "longitude",
    "room_type",
    "calculated_host_listings_count",
    "reviews_per_month"
]

df2 = df.drop(columns=drop_cols).copy()

print("Columns kept:")
print(df2.columns.tolist())
print("\nPreview (5 rows):")
print(df2.head().to_string(index=False))


Columns kept:
['id', 'name', 'host_id', 'neighbourhood_group', 'price', 'minimum_nights', 'number_of_reviews', 'last_review', 'availability_365']

Preview (5 rows):
  id                                             name  host_id neighbourhood_group  price  minimum_nights  number_of_reviews last_review  availability_365
2539               Clean & quiet apt home by the park     2787            Brooklyn    149               1                  9  2018-10-19               365
2595                            Skylit Midtown Castle     2845           Manhattan    225               1                 45  2019-05-21               355
3647              THE VILLAGE OF HARLEM....NEW YORK !     4632           Manhattan    150               3                  0         NaN               365
3831                  Cozy Entire Floor of Brownstone     4869            Brooklyn     89               1                270  2019-07-05               194
5022 Entire Apt: Spacious Studio/Loft by central park     71

## 4) Clean rows (missing values + duplicates)

For promotion targeting, I want records with complete data and no duplicates.


In [6]:
# Remove rows with missing fields (based on remaining columns)
df2 = df2.dropna().copy()

# Remove duplicate rows (full-row duplicates)
df2 = df2.drop_duplicates().copy()

print("After cleaning:")
df2.info()


After cleaning:
<class 'pandas.core.frame.DataFrame'>
Index: 38837 entries, 0 to 48852
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id                   38837 non-null  int64 
 1   name                 38837 non-null  object
 2   host_id              38837 non-null  int64 
 3   neighbourhood_group  38837 non-null  object
 4   price                38837 non-null  int64 
 5   minimum_nights       38837 non-null  int64 
 6   number_of_reviews    38837 non-null  int64 
 7   last_review          38837 non-null  object
 8   availability_365     38837 non-null  int64 
dtypes: int64(6), object(3)
memory usage: 3.0+ MB


## 5) Apply promotion rules to create the shortlist

Now I filter the cleaned dataset using the campaign requirements.


In [7]:
# Convert last_review to datetime (helps sorting and any future time analysis)
df2["last_review"] = pd.to_datetime(df2["last_review"], errors="coerce")

promotion = df2.query(
    "price <= 100 & minimum_nights <= 2 & number_of_reviews >= 100 & availability_365 >= 180"
).copy()

print("Promotion shortlist info:")
promotion.info()

print("\nPromotion shortlist preview (20 rows):")
print(promotion.head(20).to_string(index=False))


Promotion shortlist info:
<class 'pandas.core.frame.DataFrame'>
Index: 554 entries, 3 to 42075
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id                   554 non-null    int64         
 1   name                 554 non-null    object        
 2   host_id              554 non-null    int64         
 3   neighbourhood_group  554 non-null    object        
 4   price                554 non-null    int64         
 5   minimum_nights       554 non-null    int64         
 6   number_of_reviews    554 non-null    int64         
 7   last_review          554 non-null    datetime64[ns]
 8   availability_365     554 non-null    int64         
dtypes: datetime64[ns](1), int64(6), object(2)
memory usage: 43.3+ KB

Promotion shortlist preview (20 rows):
   id                                              name  host_id neighbourhood_group  price  minimum_nights  number_of_reviews last_revi

## 6) Sort and export deliverables

To make the shortlist easier to scan, I sort by:
1) borough (`neighbourhood_group`)  
2) most recent review (`last_review`)  
3) price (for a tighter “best deal” ranking within boroughs)


In [8]:
# Sorted by borough, then most recent review, then price
promotion_sorted = promotion.sort_values(
    by=["neighbourhood_group", "last_review", "price"],
    ascending=[True, False, True]
).copy()

promotion_sorted.to_csv("nyc_airbnb_promotion_shortlist.csv", index=False)
print("\nSaved: nyc_airbnb_promotion_shortlist.csv")

print("\nTop 10 after sorting:")
print(promotion_sorted.head(10).to_string(index=False))


Saved: nyc_airbnb_promotion_shortlist.csv

Top 10 after sorting:
      id                                               name  host_id neighbourhood_group  price  minimum_nights  number_of_reviews last_review  availability_365
   44096                                   Room with a View   190409               Bronx     40               1                219  2019-07-04               353
12345615                    Natural light comfortable room! 52196858               Bronx     59               1                120  2019-07-02               336
12774885           Spacious, cozy bedroom in a private home 69507287               Bronx     40               1                151  2019-07-01               338
11147993                     A SUPERB One Bedroom Apartment 57954654               Bronx     85               2                147  2019-07-01               304
   54626                     Cozy bedroom by Yankee Stadium   190409               Bronx     45               1                13

## 7) Borough distribution (how the shortlist is spread across NYC)

This gives a quick sense of where the promotion-ready listings are concentrated.
I export this as a text file for an easy “copy/paste into report” output.


In [9]:
borough_counts = promotion_sorted["neighbourhood_group"].value_counts()

print("\nListings per borough (shortlist):")
print(borough_counts)

# Overwrites each run (use mode="a" if you want to append)
with open("neighbourhood_counts.txt", "w") as f:
    print(borough_counts.to_string(), file=f)

print("\nSaved: neighbourhood_counts.txt")


Listings per borough (shortlist):
neighbourhood_group
Brooklyn         231
Manhattan        143
Queens           133
Bronx             29
Staten Island     18
Name: count, dtype: int64

Saved: neighbourhood_counts.txt


# Bonus Analysis: Luxury units in the two most popular boroughs

Airbnb also wants a quick look at **luxury listings** in the two boroughs that typically dominate NYC supply:
- Brooklyn
- Manhattan

Luxury definition used here:
- **Price:** $1000/night or more
- **Minimum stay:** 1–2 nights

Goal:
Count how many listings match, and preview the top results.


In [10]:
# Focus boroughs: Brooklyn + Manhattan
luxury_df = pd.read_csv("AB_NYC_2019.csv")

luxury_df = luxury_df.dropna().drop_duplicates().copy()

luxury_filtered = luxury_df.query("price >= 1000 & minimum_nights <= 2").copy()
luxury_filtered = luxury_filtered.query(
    'neighbourhood_group in ["Brooklyn", "Manhattan"]'
).copy()

# Sort by borough then price (highest first)
luxury_filtered = luxury_filtered.sort_values(
    by=["neighbourhood_group", "price"],
    ascending=[True, False]
).copy()

print("\nLuxury shortlist info:")
luxury_filtered.info()

print("\nLuxury preview (10 rows):")
print(luxury_filtered.head(10).to_string(index=False))

luxury_filtered.to_csv("nyc_airbnb_luxury_brooklyn_manhattan.csv", index=False)
print("\nSaved: nyc_airbnb_luxury_brooklyn_manhattan.csv")


Luxury shortlist info:
<class 'pandas.core.frame.DataFrame'>
Index: 61 entries, 4377 to 45610
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              61 non-null     int64  
 1   name                            61 non-null     object 
 2   host_id                         61 non-null     int64  
 3   host_name                       61 non-null     object 
 4   neighbourhood_group             61 non-null     object 
 5   neighbourhood                   61 non-null     object 
 6   latitude                        61 non-null     float64
 7   longitude                       61 non-null     float64
 8   room_type                       61 non-null     object 
 9   price                           61 non-null     int64  
 10  minimum_nights                  61 non-null     int64  
 11  number_of_reviews               61 non-null     int64  
 12  last_review  

# Conclusion

This analysis produced a promotion-ready shortlist of NYC Airbnb listings based on clear business rules:
low nightly price, short minimum stay, strong review signal, and high availability.

Deliverables saved to the folder:
- `nyc_airbnb_promotion_shortlist.csv` — final promotion candidates (sorted)
- `neighbourhood_counts.txt` — borough breakdown of shortlisted listings
- `nyc_airbnb_luxury_brooklyn_manhattan.csv` — luxury listings (Brooklyn + Manhattan)
